In [1]:
# !mkdir -p ./data
# !wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_semantics.zip
# !unzip data_semantics.zip -d ./data/

In [2]:
# !pip install albumentations scikit-image opencv-python

### 문제 정의(가정)
1. epoch가 많으면 모델의 성능이 좋은가?(epoch을 모두 완료한 모델 기준)    
    - epoch 50번 학습했을 때와 epoch 100번 학습했을 때 차이를 보면 epoch가 많아야 한다는 것은 분명해 보인다. 하지만 많다는 기준은 모호하고 정확한 기준을 세우기 어렵고 모델의 성능 지표에서 우선도가 낮다고 판단해 최소 100번의 학습을 다음 모델을 학습하고자 결정함  
    [epoch50 img1](./img/Loss_Epoch/E50_003.png), [epoch50 img2](./img/Loss_Epoch/E50_029.png)    
    [epoch100 img1](./img/Loss_Epoch/E100_003.png), [epoch100 img2](./Loss_Epoch/img/E100_029.png) 

2. Loss가 낮으면 모델의 성능이 좋은가?  
    - UNetV2의 Validation 오버레이 결과를 비교해 봤을 때 대부분의 결과에서 Best Loss와 Last Loss가 비슷한 양상을 보이고 일부 결과에서 간간이 차이가 보이긴 하지만 이를 통해 Loss가 낮으면 모델의 성능 좋은가에 대한 결정을 내릴 수 없을 거 같다. 
    > 내가 봤을 땐 Best Loss가 조금 더 좋아보여서 이후 테스크는 Best Loss로 비교하도록 하겠다.  
    [Best Loss img1](./img/Loss_Epoch/bestLoss_026.png) [Best Loss img2](./img/Loss_Epoch/bestLoss_002.png)  
    [Last Loss img1](./img/Loss_Epoch/bestLoss_026.png) [Last Loss img2](./img/Loss_Epoch/bestLoss_002.png) 

3. 도로인지 아닌지의 테스크로 도로 영역을 구분하기 쉬운가? [UNetV0(BCE) UNetV1(FocalLoss)]   
    - 도로 판별을 위한 BCELoss와 비도로 확신 영역의 학습 효율을 높이는 Focal Loss를 병행하여, 복잡한 환경에서도 도로 경계를 정밀하게 구분할 수 있는 세그멘테이션 모델을 구축했습니다.  
    [BCE img1](./img/BCEFOCAL/train/bce1.png)  [Focal img1](./img/BCEFOCAL/train/focal1.png)   
    [BCE img2](./img/BCEFOCAL/test/bce7.png)  [Focal img2](./img/BCEFOCAL/test/focal7.png)   
    > Focal Loss의 장점이 Focal img2에서 확인 할 수 있다 도로가 확실히 아닌 영역(하늘)을 배제하고 학습할 수 있다는 장점이 있다.

4. 기존 시맨틱 레이블의 유사 색상 배치로 인해 발생하는 도로-인도 간 경계 모호성을 해결하기 위해, semantic_rgb를 활용하여 클래스 간 대비를 극대화하는 가설을 설정하였습니다. 이를 통해 레이블의 특징 추출(Feature Extraction) 효율을 높이고, 경계 영역에서의 오분류를 최소화하는 강건한(Robust) 모델 생성을 목표로 합니다  
    -  semantic_rgb를 활용한 모델(v2)가 대부분의 결과에서 나은 모습을 확인할 수 있었으나 아직도 도로의 경계선(사이드), 그림자, 장애물을 구별하지 못하는 것을 확인할 수 있다. 
    - UNet v2 img2를 보면 v1 모델의 성능이 낮은 것을 확인할 수 있는데 이것을 생각해보면 저 영역을 모델이 학습할 때 train 데이터와 증강 방법은 동일하기 때문에 레이블이 모호했거나 학습 과정(역전파)이 잘못되었다고 생각한다.  
    [UNet v1 img1](./img/GRAYRGB/train/gray.png), [UNet v2 img1](./img/GRAYRGB/train/rgb6_FOCAL_FP.png)  
    [UNet v1 img2](./img/GRAYRGB/test/g4.png), [UNet v2 img2](./img/GRAYRGB/train/r6.png)  
    [두 모델에서 정확하지 않은 결과](./img/GRAYRGB/test/FP4.png)

5. UNet과 UNet++의 성능지표로 어떤 것을 사용해야 하는가?
 - Validation은 IoU를 구할 수 있기 때문에 정량적인 평가가 가능하다 하지만, Test 데이터는 레이블이 없기 때문에 IoU를 구할 수 없다 따라서 정성적인 평가가 필요하다. 
 - 이번 프로젝트에선 두 모델의 예측을 비교해 겹치지 않는 부분을 다른 색으로 표현 어떤 모델이 더 도로 영역을 잘 나타내는 가를 확인하고자 한다.  
 - UNet++의 연구를 많이 진행하지 못해서 UNet보다 더 좋은 가에 대한 답을 하기 어려울 거 같다. 여러 결과를 확인해보면 각 모델에서 잘 찾은 부분이나 못 찾은 부분이 많이 보였기 때문에
 이러한 결과가 모델 구조, 데이터셋, 학습과정, 하이퍼파라미터 튜닝, 레이블, 증강, 다양한 기법 도입 등을 많은 요인이 있기에 어떤 모델이 더 나은가에 대한 답을 하기 어려울 거 같다.
 [UNet v2 img1](./img/UNetUNetPP/train/UN3.png), [UNet++ img1](./img/UNetUNetPP/train/UNP1.png)  
 [UNet v2 img2](./img/UNetUNetPP/test/UN3.png), [UNet++ img2](./img/UNetUNetPP/train/UNP6.png)  
 [두 모델에서 정확하지 않은 결과](./img/UNetUNetPP/test/FP1.png)

### 회고
아쉽다.... 여기서 더 나아가서 도로 영역(레이블)에서 모델의 예측을 빼고 모델을 추가적으로 학습하는 ACoL을 구현하고 싶었지만 시간 부족해서 구현하지 못한 것이 너무 아쉽고, 도로 영역만 구별하는 테스크(한 개만 찾는 것)가 더 어려울 지 아니면 여러 객체를 동시에 구별하는 테스크(멀티 클래스)가 더 어려울 지 잘 모르겠다...
내 생각은 멀티 클래스가 더 쉬울 것으로 생각되긴 하지만 어렵다. 이번 프로젝트도 시간이 너무 부족하고 해야할 건 너무 많고 프로젝트도 어렵다... 

### 시맨틱 세그멘테이션 데이터셋

In [3]:
#필요한 라이브러리를 로드합니다.
import os
import cv2
import math
import numpy as np
from tqdm.auto import tqdm

from PIL import Image
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
from glob import glob

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from albumentations import (
    Compose, HorizontalFlip, Affine, Resize, 
    CropNonEmptyMaskIfExists, RandomShadow, RandomBrightnessContrast, 
    RandomGamma, OneOf, GaussNoise, MultiplicativeNoise, RandomSizedCrop, ShiftScaleRotate
)
print('=3')

=3


In [4]:
epoch = 100
batch_size = 8
data_dir = './data/training'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 저장 경로
save_dir = './models'

if save_dir: 
    os.makedirs(save_dir, exist_ok=True)

#### 증강에 따라 모델의 성능이 달라지는가(도로의 영역을 잘 찾는가?)

In [5]:
# def build_augmentation(is_train=True):
#     img_interp = cv2.INTER_LANCZOS4 
    
#     if is_train:
#         return Compose([
#             HorizontalFlip(p=0.5),
#             RandomSizedCrop(
#                 min_max_height=(300, 370),
#                 w2h_ratio=0.5, # 도로 형체 보존을 위해 0.5 권장
#                 size=(224, 224),
#                 p=0.3,
#                 interpolation=img_interp,           # 이미지: 고화질 보간
#                 mask_interpolation=cv2.INTER_NEAREST # 마스크: 절대 값 유지
#             ),
#             Resize(
#                 width=224,
#                 height=224,
#                 interpolation=img_interp,
#                 mask_interpolation=cv2.INTER_NEAREST # 필수
#             )
#         ])
#     return Compose([
#         Resize(
#             width=224,
#             height=224,
#             interpolation=img_interp,
#             mask_interpolation=cv2.INTER_NEAREST # 필수
#         )
#     ])

In [6]:
def build_augmentation_v2(is_train=True):
    img_interp = cv2.INTER_LANCZOS4 
    
    if is_train:
        return Compose([
            # 1. 도로 중심 크롭 
            CropNonEmptyMaskIfExists(
                height=370, 
                width=740, 
                p=1.0
            ),

            # 2. 기하학적 변형 (Affine 최신 표준)
            HorizontalFlip(p=0.5),
            Affine(
                translate_percent={"x": (-0.03, 0.03), "y": (-0.03, 0.03)}, 
                scale=(0.95, 1.05), 
                rotate=(-3, 3),
                border_mode=cv2.BORDER_REPLICATE, 
                p=0.4
            ),

            # 3. 환경 대응 
            OneOf([
                RandomShadow(num_shadows_limit=(1, 2), shadow_dimension=5, p=1.0),
                RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0),
                RandomGamma(gamma_limit=(80, 120), p=1.0),
            ], p=0.6),

            # 4. 노이즈
            GaussNoise(std_range=(0.0, 0.01), p=0.2),

            Resize(width=224, height=224, interpolation=img_interp, mask_interpolation=cv2.INTER_NEAREST)
        ])
    
    return Compose([
        Resize(width=224, height=224, interpolation=img_interp, mask_interpolation=cv2.INTER_NEAREST)
    ])

In [7]:
# def build_augmentation_v3(is_train=True):
#     img_interp = cv2.INTER_LANCZOS4 
#     if is_train:
#         return Compose([
#             HorizontalFlip(p=0.5),
#             # 과한 Crop 대신, 안정적인 위치 변화를 줍니다.
#             ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=5, p=0.5),
#             # 조명 조건 대응
#             RandomBrightnessContrast(p=0.5),
#             # 최종 사이즈 결정 (왜곡은 여기서 어쩔 수 없이 발생하지만, 앞단에서 최소화함)
#             Resize(width=224, height=224,
#                 interpolation=img_interp,
#                 mask_interpolation=cv2.INTER_NEAREST # 필수
#                 )
#         ])
#     return Compose([
#         Resize(width=224, height=224,
#                 interpolation=img_interp,
#                 mask_interpolation=cv2.INTER_NEAREST # 필수
#                 )
#     ])

In [8]:
# # 증강이 적절하게 되는가(확인용)
# augmentation_train =build_augmentation_v2()
# augmentation_test = build_augmentation_v2(is_train=False)
# # 1. 마스크 파일 경로 목록 생성 (image_2 폴더를 semantic 폴더로 변경)
# # 데이터셋 구조에 따라 경로를 수정하세요. (예: image_2 -> semantic)
# mask_images = sorted(glob(os.path.join(data_dir, "semantic", "*.png")))
# input_images = sorted(glob(os.path.join(data_dir, "image_2", "*.png")))

# plt.figure(figsize=(10, 15))

# for i in range(5):
#     # 이미지와 마스크 로드
#     image = imread(input_images[i])
#     mask = imread(mask_images[i]) # 해당 이미지의 정답 마스크 로드
    
#     # [핵심] image와 mask를 동시에 딕셔너리에 담기
#     # CropNonEmptyMaskIfExists가 이 mask를 보고 도로 영역을 찾아냅니다.
#     image_data = {"image": image, "mask": mask}
    
#     # 2. 증강 적용
#     # Test용 (단순 Resize)
#     resized = augmentation_test(**image_data)
#     # Train용 (도로 중심 크롭 + 그림자 등 적용)
#     processed = augmentation_train(**image_data)
    
#     # --- 시각화 ---
#     # 왼쪽: 단순 Resize 결과
#     plt.subplot(5, 2, 2*i+1)
#     plt.imshow(resized["image"])
#     plt.title(f"Original (Resized) {i+1}")
#     plt.axis('off')
    
#     # 오른쪽: 도로 중심 크롭 및 증강 결과
#     plt.subplot(5, 2, 2*i+2)
#     plt.imshow(processed["image"])
#     plt.title(f"Road-Centered Aug {i+1}")
#     plt.axis('off')

# plt.tight_layout()
# plt.show()

---
#### KittiDataset

In [9]:
class KittiDataset(Dataset):
    '''
    KittiDataset은 PyTorch의 Dataset을 상속받습니다.
    우리가 KittiDataset을 원하는 방식으로 preprocess하기 위해서 Dataset을 커스텀하여 사용합니다.
    '''
    def __init__(self,
                 dir_path,
                 img_size=(224, 224, 3),
                 output_size=(224, 224),
                 is_train=True,
                 augmentation=None):
        '''
        dir_path: dataset의 directory path입니다.
        img_size: preprocess에 사용할 입력이미지의 크기입니다.
        output_size: ground_truth를 만들어주기 위한 크기입니다.
        is_train: 이 Dataset이 학습용인지 테스트용인지 구분합니다.
        augmentation: 적용하길 원하는 augmentation 함수를 인자로 받습니다.
        '''
        self.dir_path = dir_path
        self.is_train = is_train
        self.augmentation = augmentation
        self.img_size = img_size
        self.output_size = output_size

        # load_dataset()을 통해 kitti dataset의 경로에서 라벨과 이미지를 확인합니다.
        self.data = self.load_dataset()

    def load_dataset(self):
        # kitti dataset에서 필요한 정보(이미지 경로 및 라벨)를 directory에서 확인하고 로드하는 함수입니다.
        input_images = sorted(glob(os.path.join(self.dir_path, "image_2", "*.png")))
        label_images = sorted(glob(os.path.join(self.dir_path, "semantic", "*.png")))

        assert len(input_images) == len(label_images)
        data = list(zip(input_images, label_images))

        if self.is_train:
            return data[:-40]
        return data[-40:]

    def __len__(self):
        # Dataset의 length로서 전체 dataset 크기를 반환합니다.
        return len(self.data)

    def __getitem__(self, index):
        # 입력과 출력을 만듭니다.
        # 입력은 resize 및 augmentation이 적용된 input image이고
        # 출력은 semantic label입니다.
        input_img_path, output_path = self.data[index]

        _input = cv2.cvtColor(imread(input_img_path), cv2.COLOR_BGR2RGB)
        _output = imread(output_path)

        # 특정 라벨을 이진 마스크로 변환(중요)
        _output = (_output == 7).astype(np.uint8) * 1

        data = {
            "image": _input,
            "mask": _output,
        }

        if self.augmentation:
            augmented = self.augmentation(**data)
            _input = augmented["image"]  
            _output = augmented["mask"]

        if _input.max() > 1.0: 
            _input = _input / 255.0 # Normalize

        # target 차원 확장 (H, W) -> (1, H, W)
        # Albumentations를 거치면 차원이 사라질 수 있으므로 마지막에 하는 것이 안전
        if _output.ndim == 2:
            _output = np.expand_dims(_output, axis=0)

        return (
            torch.tensor(_input, dtype=torch.float32).permute(2, 0, 1),  # (H, W, C) → (C, H, W)
            torch.tensor(_output, dtype=torch.float32)  # (1, H, W) 형식 유지
        )

    def shuffle_data(self):
        # 한 epoch가 끝나면 실행되는 함수입니다. 학습 중인 경우에 데이터를 random shuffle합니다.
        if self.is_train:
            np.random.shuffle(self.data)

In [10]:
# semantic_rgb에서 도로 영역(1) 외 나머지 객체를 0으로 처리
class KittiDatasetV2(Dataset):
    def __init__(self,
                 dir_path,
                 img_size=(224, 224, 3),
                 output_size=(224, 224),
                 is_train=True,
                 augmentation=None):
        self.dir_path = dir_path
        self.is_train = is_train
        self.augmentation = augmentation
        self.img_size = img_size
        self.output_size = output_size

        # 데이터 경로 로드
        self.data = self.load_dataset()

    def load_dataset(self):
        # image_2 폴더와 semantic 폴더에서 파일 리스트를 가져옵니다.
        input_images = sorted(glob(os.path.join(self.dir_path, "image_2", "*.png")))
        label_images = sorted(glob(os.path.join(self.dir_path, "semantic_rgb", "*.png")))

        assert len(input_images) == len(label_images), "이미지와 라벨의 개수가 일치하지 않습니다."
        data = list(zip(input_images, label_images))

        if self.is_train:
            return data[:-40]
        return data[-40:]
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_img_path, output_path = self.data[index]

        # 1. 이미지 및 RGB 라벨 로드
        # imread 대신 cv2를 사용하여 색상 채널을 명확히 관리합니다.
        _input = cv2.imread(input_img_path)
        _input = cv2.cvtColor(_input, cv2.COLOR_BGR2RGB)
        
        _output_rgb = cv2.imread(output_path)
        _output_rgb = cv2.cvtColor(_output_rgb, cv2.COLOR_BGR2RGB)

        # 2. [Semantic v2 핵심] RGB 색상 매칭을 통한 이진 마스크 생성
        # KITTI/Cityscapes 표준 도로 색상: 보라색 [128, 64, 128]
        # 사용자님의 데이터에 맞춰 이 값을 수정할 수 있습니다.
        road_color = [128, 64, 128] 
        _output = np.all(_output_rgb == road_color, axis=-1).astype(np.uint8)

        data = {
            "image": _input,
            "mask": _output,
        }

        # 3. Augmentation 적용
        if self.augmentation:
            augmented = self.augmentation(**data)
            _input = augmented["image"]  
            _output = augmented["mask"]

        # 4. 정규화 (Normalization)
        if _input.max() > 1.0: 
            _input = _input / 255.0

        # 5. 차원 맞추기 (H, W) -> (1, H, W)
        if _output.ndim == 2:
            _output = np.expand_dims(_output, axis=0)

        # 6. Tensor 변환 및 채널 순서 변경 (H,W,C -> C,H,W)
        return (
            torch.tensor(_input, dtype=torch.float32).permute(2, 0, 1),
            torch.tensor(_output, dtype=torch.float32)
        )

    def shuffle_data(self):
        if self.is_train:
            np.random.shuffle(self.data)

In [11]:
# Augmentation 설정s
augmentation = build_augmentation_v2()
val_preproc = build_augmentation_v2(is_train=False)

In [12]:
train_dataset = KittiDataset(
    data_dir,
    augmentation=augmentation,
    is_train=True
)

val_dataset = KittiDataset(
    data_dir,
    augmentation=val_preproc,
    is_train=False
)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [13]:
train_dataset_v2 = KittiDatasetV2(
    data_dir,
    augmentation=augmentation,
    is_train=True
)

val_dataset_v2 = KittiDatasetV2(
    data_dir,
    augmentation=val_preproc,
    is_train=False
)

# DataLoader 설정
train_loader_v2 = DataLoader(train_dataset_v2, batch_size=batch_size, shuffle=True)
val_loader_v2 = DataLoader(val_dataset_v2, batch_size=batch_size, shuffle=False)

In [14]:
# 두 데이터셋의 검증용 데이터 경로를 가져와 비교
val_images = [d[0] for d in val_dataset.data] # KittiDataset (is_train=False일 때)
val_images_v2 = [d[0] for d in val_dataset_v2.data] # KittiDatasetV2 (is_train=False일 때)

# 첫 번째 파일명이 같은지 확인
print(f"V1 첫번째 이미지: {val_images[5]}")
print(f"V2 첫번째 이미지: {val_images_v2[5]}")

# 전체가 완벽히 일치하는지 확인
print(f"두 데이터셋의 이미지가 완전히 같은가? : {val_images == val_images_v2}")

V1 첫번째 이미지: ./data/training/image_2/000165_10.png
V2 첫번째 이미지: ./data/training/image_2/000165_10.png
두 데이터셋의 이미지가 완전히 같은가? : True


### 시맨틱 세그멘테이션 모델

In [15]:
class Unet_block(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.conv2 = nn.Conv2d(mid_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)        
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        return out

In [16]:
class UNet(nn.Module):
    def __init__(self, input_channels=3, output_channels=1):
        super(UNet, self).__init__()

        # [Encoder] - Unet_block 사용
        self.enc1 = Unet_block(input_channels, 64, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = Unet_block(64, 128, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = Unet_block(128, 256, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.enc4 = Unet_block(256, 512, 512)
        self.pool4 = nn.MaxPool2d(2)

        # [Bottleneck]
        self.bottleneck = Unet_block(512, 1024, 1024)
        self.dropout = nn.Dropout(0.5)

        # [Decoder]
        self.up6 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec6 = Unet_block(1024, 512, 512) # Cat(512+512) -> 512
        self.up7 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec7 = Unet_block(512, 256, 256)
        self.up8 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec8 = Unet_block(256, 128, 128)
        self.up9 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec9 = Unet_block(128, 64, 64)

        self.final = nn.Conv2d(64, output_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        c1 = self.enc1(x)
        p1 = self.pool1(c1)
        c2 = self.enc2(p1)
        p2 = self.pool2(c2)
        c3 = self.enc3(p2)
        p3 = self.pool3(c3)
        c4 = self.enc4(p3)
        p4 = self.pool4(c4)

        # Bottleneck
        c5 = self.bottleneck(p4)
        c5 = self.dropout(c5)

        # Decoder
        u6 = self.up6(c5)
        u6 = torch.cat([u6, c4], dim=1)
        c6 = self.dec6(u6)

        u7 = self.up7(c6)
        u7 = torch.cat([u7, c3], dim=1)
        c7 = self.dec7(u7)

        u8 = self.up8(c7)
        u8 = torch.cat([u8, c2], dim=1)
        c8 = self.dec8(u8)

        u9 = self.up9(c8)
        u9 = torch.cat([u9, c1], dim=1)
        c9 = self.dec9(u9)

        output = self.final(c9)
        return torch.sigmoid(output)

In [17]:
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)

In [18]:
class Nested_UNet(nn.Module):
    def __init__(self, input_channels=3, output_channels=1, deep_supervision=False):
        super().__init__()

        # 모델 크기 설정 (원할 경우 [64, 128, 256, 512, 1024]로 변경 가능)
        nb_filter = [64, 128, 256, 512, 1024]
        
        self.deep_supervision = deep_supervision
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        # --- Backbone (DownSampling) ---
        self.conv0_0 = Unet_block(input_channels, nb_filter[0], nb_filter[0])
        self.conv1_0 = Unet_block(nb_filter[0], nb_filter[1], nb_filter[1])
        self.conv2_0 = Unet_block(nb_filter[1], nb_filter[2], nb_filter[2])
        self.conv3_0 = Unet_block(nb_filter[2], nb_filter[3], nb_filter[3])
        self.conv4_0 = Unet_block(nb_filter[3], nb_filter[4], nb_filter[4])

        # --- Upsampling & Dense skip connections ---
        # N to 1 skip (X^0,1 | X^1,1 | X^2,1 | X^3,1)
        self.conv0_1 = Unet_block(nb_filter[0] + nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_1 = Unet_block(nb_filter[1] + nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_1 = Unet_block(nb_filter[2] + nb_filter[3], nb_filter[2], nb_filter[2])
        self.conv3_1 = Unet_block(nb_filter[3] + nb_filter[4], nb_filter[3], nb_filter[3])
       
        # N to 2 skip (X^0,2 | X^1,2 | X^2,2)
        self.conv0_2 = Unet_block(nb_filter[0]*2 + nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_2 = Unet_block(nb_filter[1]*2 + nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_2 = Unet_block(nb_filter[2]*2 + nb_filter[3], nb_filter[2], nb_filter[2])

        # N to 3 skip (X^0,3 | X^1,3)
        self.conv0_3 = Unet_block(nb_filter[0]*3 + nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_3 = Unet_block(nb_filter[1]*3 + nb_filter[2], nb_filter[1], nb_filter[1])

        # N to 4 skip (X^0,4)
        self.conv0_4 = Unet_block(nb_filter[0]*4 + nb_filter[1], nb_filter[0], nb_filter[0])

        # --- Output Layers ---
        if self.deep_supervision:
            self.output1 = nn.Conv2d(nb_filter[0], output_channels, kernel_size=1)
            self.output2 = nn.Conv2d(nb_filter[0], output_channels, kernel_size=1)
            self.output3 = nn.Conv2d(nb_filter[0], output_channels, kernel_size=1)
            self.output4 = nn.Conv2d(nb_filter[0], output_channels, kernel_size=1)
        else:
            self.output = nn.Conv2d(nb_filter[0], output_channels, kernel_size=1)

        # 모델 파라미터 Kaiming 초기화 적용
        self.apply(init_weights)

    def forward(self, x):
        # 대각선(Diagonal) 흐름으로 계산하여 메모리 할당 효율성 극대화
        
        # Level 0 & 1
        x0_0 = self.conv0_0(x)               
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], dim=1))
        
        # Level 2
        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], dim=1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], dim=1))

        # Level 3
        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], dim=1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], dim=1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], dim=1))

        # Level 4
        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], dim=1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], dim=1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], dim=1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], dim=1))

        # --- Return (Raw Logits 반환) ---
        if self.deep_supervision:
            output1 = self.output1(x0_1)
            output2 = self.output2(x0_2)
            output3 = self.output3(x0_3)
            output4 = self.output4(x0_4)
            # 리스트 형태로 개별 Logit을 반환 (Loss 계산 시 각각 더해줌)
            return [torch.sigmoid(output1), 
                    torch.sigmoid(output2), 
                    torch.sigmoid(output3), 
                    torch.sigmoid(output4)]
        else:
            # 최종 아웃풋 Logit만 반환
            output = self.output(x0_4)
            return torch.sigmoid(output)

### Train Validation

In [19]:
def calculate_metrics(pred, target, threshold=0.5):
    # pred: (B, 1, H, W), target: (B, 1, H, W)
    pred = (torch.sigmoid(pred) > threshold).float()
    target = target.float()

    # 각 이미지별로 Intersection과 Union 계산 (dim 1, 2, 3에 대해 합산)
    # 결과는 (Batch_size,) 형태의 벡터가 됨
    intersection = (pred * target).sum(dim=(1, 2, 3))
    union = pred.sum(dim=(1, 2, 3)) + target.sum(dim=(1, 2, 3)) - intersection

    # 이미지별 IoU 계산 (0으로 나누기 방지)
    iou_per_image = (intersection + 1e-6) / (union + 1e-6)
    
    # 이미지별 Pixel Accuracy 계산
    acc_per_image = (pred == target).float().mean(dim=(1, 2, 3))
    
    # 배치 평균값 반환
    return iou_per_image.mean().item(), acc_per_image.mean().item()

In [20]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.5, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # 1. BCE Loss 계산 (Logits을 입력으로 받음)
        # targets는 [B, 1, H, W] 형태이며 float 타입이어야 함
        bce_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        
        # 2. pt (정답에 대한 확률) 계산
        pt = torch.exp(-bce_loss) 
        
        # 3. Focal Loss 공식 적용: FL(pt) = -alpha * (1 - pt)^gamma * log(pt)
        focal_loss = self.alpha * (1 - pt)**self.gamma * bce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [21]:
def train_model(model, train_loader, val_loader, criterion, device, num_epochs, save_dir, lr=1e-4,model_type='_unet'):
    best_loss = float('inf')
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)

    steps_per_epoch = len(train_loader)
    # scheduler 정의 (patience=5는 5에폭 동안 개선 없을 시 LR 감소)
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=1e-3,              # 도달하고 싶은 최대 학습률 (보통 초기 lr의 10배 정도)
        steps_per_epoch=steps_per_epoch,
        epochs=num_epochs,
        pct_start=0.3,            # 전체 학습 중 30% 구간을 워밍업(학습률 상승)에 할당
        anneal_strategy='cos',    # 코사인 곡선으로 학습률 하강
        div_factor=25.0,          # 시작 학습률 = max_lr / 25.0
        final_div_factor=10000.0  # 최종 학습률 = 시작 학습률 / 10000.0 (거의 0에 가깝게)
    )

    # 1. 에폭 단위 진행바
    epoch_pbar = tqdm(range(num_epochs), desc="Total Epochs")

    for epoch in epoch_pbar:
        # --- [Training Phase] ---
        model.train()
        train_loss, train_iou, train_acc = 0.0, 0.0, 0.0

        if hasattr(train_loader.dataset, 'shuffle_data'):
            train_loader.dataset.shuffle_data()

        # 2. 학습 배치 단위 진행바 (leave=False로 에폭 종료 시 정리)
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]", leave=False)
        for inputs, targets in train_pbar:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            
            if isinstance(outputs, list):
                loss = sum([criterion(o, targets.float()) for o in outputs])
                final_output = outputs[-1] 
            else:
                loss = criterion(outputs, targets.float())
                final_output = outputs

            loss.backward()
            optimizer.step()

            # 학습률 스케줄러 업데이트
            scheduler.step()

            # 지표 계산 및 업데이트
            running_iou, running_acc = calculate_metrics(final_output, targets)
            train_loss += loss.item()
            train_iou += running_iou
            train_acc += running_acc
            
            # 실시간 배치 정보 표시
            train_pbar.set_postfix(loss=f"{loss.item():.4f}", iou=f"{running_iou:.4f}")

        # --- [Validation Phase] ---
        model.eval()
        val_loss, val_iou, val_acc = 0.0, 0.0, 0.0

        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]", leave=False)
        with torch.no_grad():
            for inputs, targets in val_pbar:
                inputs, targets = inputs.to(device), targets.to(device)
                
                outputs = model(inputs)
                
                if isinstance(outputs, list):
                    v_loss = sum([criterion(o, targets.float()) for o in outputs])
                    final_output = outputs[-1]
                else:
                    v_loss = criterion(outputs, targets.float())
                    final_output = outputs
                
                v_iou, v_acc = calculate_metrics(final_output, targets)
                val_loss += v_loss.item()
                val_iou += v_iou
                val_acc += v_acc
                
                val_pbar.set_postfix(loss=f"{v_loss.item():.4f}", iou=f"{v_iou:.4f}")

        avg_val_loss = val_loss / len(val_loader) if len(val_loader) > 0 else 0
        avg_val_iou = val_iou / len(val_loader) if len(val_loader) > 0 else 0

        # 에폭바에 최종 결과 출력
        current_lr = optimizer.param_groups[0]['lr']
        epoch_pbar.set_postfix(v_loss=f"{avg_val_loss:.4f}", v_iou=f"{avg_val_iou:.4f}", lr=f"{current_lr:.1e}")

        # Best Model Save 
        if avg_val_loss < best_loss and avg_val_loss > 0:
            best_loss = avg_val_loss
            torch.save(model.state_dict(), f'{save_dir}/best{model_type}.pth')

    torch.save(model.state_dict(), f'{save_dir}/last{model_type}.pth')

### evaluation

In [22]:
def calculate_iou_score(target, prediction, verbose=False):
    if target.shape != prediction.shape:
        prediction = resize(prediction, target.shape, mode='constant', preserve_range=True).astype(np.uint8)

    intersection = np.logical_and(target, prediction).sum()
    union = np.logical_or(target, prediction).sum()
    iou_score = intersection / (union + 1e-7)  
    if verbose: 
        print(f"IoU : {iou_score:.6f}")
    return iou_score

In [23]:
def get_output(model, preproc, image_path, output_path, label_path = None, im_show = False):
    # 1. 모델이 현재 위치한 장치를 자동으로 확인
    # 모델 가중치의 첫 번째 파라미터를 기준으로 삼습니다.
    device = next(model.parameters()).device

    # 2. 이미지 로드 및 전처리 (사용자 기존 로직)
    origin_img = imread(image_path)
    data = {"image": origin_img}
    processed = preproc(**data)
    
    # 3. 텐서 생성 및 장치 이동 (핵심 수정 포인트)
    # / 255.0으로 정규화한 뒤, 모델과 같은 장치(device)로 보냅니다.
    input_tensor = torch.tensor(processed["image"] / 255.0, dtype=torch.float32)
    input_tensor = input_tensor.permute(2, 0, 1).unsqueeze(0).to(device) # .to(device) 추가

    # 4. 모델 추론
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)

    # 5. 후처리 (시각화를 위해 다시 CPU로 가져옴)
    # GPU에 있는 결과를 다시 넘파이로 바꾸려면 .cpu()가 필수입니다.
    prediction = (output[0].squeeze().cpu().numpy() > 0.5).astype(np.uint8) * 255
    prediction = Image.fromarray(prediction).convert('L')

    # 6. 블렌딩 및 시각화 (사용자 기존 로직 유지)
    background = Image.fromarray(origin_img).convert('RGBA')
    prediction_resized = prediction.resize((origin_img.shape[1], origin_img.shape[0])).convert('RGBA')
    blended = Image.blend(background, prediction_resized, alpha=0.5)

    blended.save(output_path)
    img_np = np.array(blended)

    if im_show:
        plt.imshow(img_np)
        plt.axis('off')
        plt.show()

    target = None
    if label_path:
        label_img = imread(label_path)
        label_processed = preproc(image=label_img)["image"]
        target = (label_processed == 7).astype(np.uint8) * 1

    return blended, np.array(prediction), target

In [24]:
def get_comparison_output(model_u, model_pp, preproc, image_path, output_path, model_type, label_path=None, idx=0):
    # 1. 모델 장치 확인
    device_u = next(model_u.parameters()).device
    device_pp = next(model_pp.parameters()).device
    
    # 2. 이미지 로드 및 채널 처리
    origin_img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if origin_img is None: return

    # 채널 수 확인 (1채널 vs 3채널)
    if origin_img.ndim == 2:
        h, w = origin_img.shape
        c = 1
        # 시각화용 배경은 3채널로 변환
        vis_background = cv2.cvtColor(origin_img, cv2.COLOR_GRAY2RGB)
    else:
        h, w, c = origin_img.shape
        vis_background = cv2.cvtColor(origin_img, cv2.COLOR_BGR2RGB)

    # 3. 모델 입력을 위한 전처리
    # Dataset 로직: 입력은 RGB여야 함
    img_rgb = cv2.cvtColor(origin_img, cv2.COLOR_BGR2RGB) if c == 3 else cv2.cvtColor(origin_img, cv2.COLOR_GRAY2RGB)
    
    processed = preproc(image=img_rgb)
    img_pre = processed["image"]
    
    # 정규화
    if img_pre.max() > 1.0:
        img_pre = img_pre / 255.0

    # 텐서 변환 (B, C, H, W)
    input_tensor = torch.tensor(img_pre, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)

    # 4. 모델 추론
    model_u.eval(); model_pp.eval()
    with torch.no_grad():
        out_u = model_u(input_tensor.to(device_u))
        if isinstance(out_u, (list, tuple)): out_u = out_u[0]
        pred_u_bin = (out_u.squeeze().cpu().numpy() > 0.5).astype(np.uint8)
        
        out_pp = model_pp(input_tensor.to(device_pp))
        if isinstance(out_pp, (list, tuple)): out_pp = out_pp[-1]
        pred_pp_bin = (out_pp.squeeze().cpu().numpy() > 0.5).astype(np.uint8)

    # 5. IoU 계산 (1채널/3채널 라벨 분기 처리)
    iou_u, iou_pp = 0.0, 0.0
    if label_path:
        label_raw = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)
        
        # 라벨 채널에 따른 이진 마스크 생성 로직 분기
        if label_raw.ndim == 3:
            # 라벨이 3채널 RGB인 경우 (특정 색상 매칭)
            label_rgb = cv2.cvtColor(label_raw, cv2.COLOR_BGR2RGB)
            road_color = [128, 64, 128] # Dataset 로직 참조
            target_bin = np.all(label_rgb == road_color, axis=-1).astype(np.uint8)
        else:
            # 라벨이 1채널인 경우 (값 7 필터링)
            target_bin = (label_raw == 7).astype(np.uint8)

        # 전처리 적용 (Resize 등)
        label_processed = preproc(image=img_rgb, mask=target_bin)
        target = label_processed["mask"].astype(np.uint8)
        if target.ndim == 3: target = np.squeeze(target, axis=0)
            
        iou_u = calculate_iou_score(target, pred_u_bin)
        iou_pp = calculate_iou_score(target, pred_pp_bin)
        print(f"[{str(idx).zfill(3)}] > {model_type[0]}: {iou_u:.4f}, {model_type[1]}: {iou_pp:.4f}")

    # 6. 시각화 마스크 생성
    m_u = cv2.resize(pred_u_bin, (w, h), interpolation=cv2.INTER_NEAREST)
    m_pp = cv2.resize(pred_pp_bin, (w, h), interpolation=cv2.INTER_NEAREST)
    
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)
    color_mask[(m_u == 1) & (m_pp == 1)] = [255, 255, 255]
    color_mask[(m_u == 1) & (m_pp == 0)] = [255, 0, 0]
    color_mask[(m_u == 0) & (m_pp == 1)] = [0, 0, 255]

    # 7. 합성 및 저장
    background_pil = Image.fromarray(vis_background).convert('RGBA')
    mask_pil = Image.fromarray(color_mask).convert('RGBA')
    blended_pil = Image.blend(background_pil, mask_pil, alpha=0.5)
    
    blended_np = np.array(blended_pil.convert('RGB'))
    header_h = 50
    final_img = np.zeros((h + header_h, w, 3), dtype=np.uint8)
    final_img[header_h:, :] = blended_np
    
    info_text = f"{model_type[0]}(Red): {iou_u:.2f}, {model_type[1]}(Blue): {iou_pp:.2f}"
    cv2.putText(final_img, info_text, (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    Image.fromarray(final_img).save(output_path)
    return Image.fromarray(final_img)

## UNet

---
#### BCDLoss  
기존 방식: 도로영역만 찾는 이진 분류

In [25]:
# # 모델 생성
# unet_v0 = UNet(input_channels=3, output_channels=1)
# unet_v0.to(device)

# # 손실 함수 및 옵티마이저 설정
# criterion = torch.nn.BCELoss()

# train_model(
#     unet_v0, train_loader, val_loader, criterion, 
#     device=device, 
#     num_epochs=epoch, 
#     save_dir=save_dir, 
#     model_type='_unet_bcd'  # 모델 타입 명시
# )

In [26]:
model_path = f"{save_dir}/best_unet_bcd.pth"
unet_v0 = UNet(input_channels=3, output_channels=1)
unet_v0.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
unet_v0.eval()  # 평가 모드로 전환

UNet(
  (enc1): Unet_block(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc2): Unet_block(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc3): Unet_block(
  

In [27]:
dir_path = './data/testing'
save_train_dir = './data/result/unetV0_train'
save_test_dir = './data/result/unetV0_test'

os.makedirs(dir_path, exist_ok=True)
os.makedirs(save_train_dir, exist_ok=True)
os.makedirs(save_test_dir, exist_ok=True)

In [28]:
# 검증 데이터는 val_dataset.data 리스트를 사용하여 라벨 경로를 함께 넘깁니다.
for i in tqdm(range(len(val_dataset.data)), desc="Validation Analysis"):
    img_path, label_path = val_dataset.data[i]

    get_output(
        unet_v0, val_preproc,
        image_path=img_path,
        output_path= f'{save_train_dir}/result_{str(i).zfill(3)}.png',
        label_path=label_path,
        im_show = False
    )

Validation Analysis:   0%|          | 0/40 [00:00<?, ?it/s]

In [29]:
# 완성한 뒤에는 시각화한 결과를 눈으로 확인해봅시다!
for i in tqdm(range(200), desc="Test Inference Progres", unit="img"):
    
    img_name = f"00{str(i).zfill(4)}_10.png"
    img_full_path = os.path.join(dir_path, "image_2", img_name)
    
    res_name = f"result_{str(i).zfill(3)}.png"
    res_full_path = os.path.join(save_test_dir, res_name)

    # 추론 실행
    get_output(
        unet_v0,
        val_preproc,
        image_path=img_full_path,
        output_path=res_full_path
    )

Test Inference Progres:   0%|          | 0/200 [00:00<?, ?img/s]

---
FocalLoss  
개선 방식: 도로영역 뿐만 아니라 도로가 아닌(배경) 영역도 찾는 이진 분류

In [30]:
# # 모델 생성
# unet_v1 = UNet(input_channels=3, output_channels=1)
# unet_v1.to(device)

# # 손실 함수 및 옵티마이저 설정
# criterion = FocalLoss(alpha=0.5, gamma=2.0)

# train_model(
#     unet_v1, train_loader, val_loader, criterion, 
#     device=device, 
#     num_epochs=epoch, 
#     save_dir=save_dir, 
#     model_type='_unet'  # 모델 타입 명시
# )

In [31]:
model_path = f"{save_dir}/best_unet.pth"
unet_v1 = UNet(input_channels=3, output_channels=1)
unet_v1.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
unet_v1.eval()  # 평가 모드로 전환

UNet(
  (enc1): Unet_block(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc2): Unet_block(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc3): Unet_block(
  

In [32]:
dir_path = './data/testing'
save_train_dir = './data/result/unetV1_train'
save_test_dir = './data/result/unetV1_test'

os.makedirs(dir_path, exist_ok=True)
os.makedirs(save_train_dir, exist_ok=True)
os.makedirs(save_test_dir, exist_ok=True)

In [33]:
# 검증 데이터는 val_dataset.data 리스트를 사용하여 라벨 경로를 함께 넘깁니다.
for i in tqdm(range(len(val_dataset.data)), desc="Validation Analysis"):
    img_path, label_path = val_dataset.data[i]

    get_output(
        unet_v1, val_preproc,
        image_path=img_path,
        output_path= f'{save_train_dir}/result_{str(i).zfill(3)}.png',
        label_path=label_path,
        im_show = False
    )

Validation Analysis:   0%|          | 0/40 [00:00<?, ?it/s]

In [34]:
# 완성한 뒤에는 시각화한 결과를 눈으로 확인해봅시다!
for i in tqdm(range(200), desc="Test Inference Progres", unit="img"):
    
    img_name = f"00{str(i).zfill(4)}_10.png"
    img_full_path = os.path.join(dir_path, "image_2", img_name)
    
    res_name = f"result_{str(i).zfill(3)}.png"
    res_full_path = os.path.join(save_test_dir, res_name)

    # 추론 실행
    get_output(
        unet_v1,
        val_preproc,
        image_path=img_full_path,
        output_path=res_full_path
    )

Test Inference Progres:   0%|          | 0/200 [00:00<?, ?img/s]

##### UNetV2: FocalLoss, KittiDatasetV2  

In [35]:
# # 모델 생성
# unet_v2 = UNet(input_channels=3, output_channels=1)
# unet_v2.to(device)

# # 손실 함수 및 옵티마이저 설정
# criterion = FocalLoss(alpha=0.5, gamma=2.0)

# train_model(
#     unet_v2, train_loader_v2, val_loader_v2, criterion,
#     device=device, 
#     num_epochs=epoch, 
#     save_dir=save_dir, 
#     model_type='_unet_v2'  # 모델 타입 명시
# )

In [36]:
model_path = f"{save_dir}/best_unet_v2.pth"
unet_v2 = UNet(input_channels=3, output_channels=1)
unet_v2.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
unet_v2.eval()  # 평가 모드로 전환

UNet(
  (enc1): Unet_block(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc2): Unet_block(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc3): Unet_block(
  

In [37]:
dir_path = './data/testing'
save_train_dir = './data/result/unet_v2_train'
save_test_dir = './data/result/unet_v2_test'

os.makedirs(dir_path, exist_ok=True)
os.makedirs(save_train_dir, exist_ok=True)
os.makedirs(save_test_dir, exist_ok=True)

In [38]:
# 검증 데이터는 val_dataset.data 리스트를 사용하여 라벨 경로를 함께 넘깁니다.
for i in tqdm(range(len(val_dataset_v2.data)), desc="Validation Analysis"):
    img_path, label_path = val_dataset_v2.data[i]

    get_output(
        unet_v2, val_preproc,
        image_path=img_path,
        output_path= f'{save_train_dir}/result_{str(i).zfill(3)}.png',
        label_path=label_path,
        im_show = False
    )

Validation Analysis:   0%|          | 0/40 [00:00<?, ?it/s]

In [39]:
# 완성한 뒤에는 시각화한 결과를 눈으로 확인해봅시다!
for i in tqdm(range(200), desc="Test Inference Progres", unit="img"):
    
    img_name = f"00{str(i).zfill(4)}_10.png"
    img_full_path = os.path.join(dir_path, "image_2", img_name)
    
    res_name = f"result_{str(i).zfill(3)}.png"
    res_full_path = os.path.join(save_test_dir, res_name)

    # 추론 실행
    get_output(
        unet_v2,
        val_preproc,
        image_path=img_full_path,
        output_path=res_full_path
    )

Test Inference Progres:   0%|          | 0/200 [00:00<?, ?img/s]

UNetV0(BCDLOSS) VS UNetV1(FocalLoss)

In [40]:
# 검증 데이터는 val_dataset.data 리스트를 사용하여 라벨 경로를 함께 넘깁니다.
compare_val_dir = os.path.join('./', 'data', 'result', 'comparison_results', 'unetv0,1_val')
os.makedirs(compare_val_dir, exist_ok=True)

for i in tqdm(range(len(val_dataset.data)), desc="Validation Analysis"):
    img_path, label_path = val_dataset.data[i]
    res_full_path = os.path.join(compare_val_dir, f"val_compare_{str(i).zfill(2)}.png")

    get_comparison_output(
        unet_v0, unet_v1, val_preproc,
        image_path=img_path,
        output_path=res_full_path,
        model_type=['UNetV0','UNetV1'],
        label_path=label_path, # 라벨 전달 -> IoU 및 텍스트 출력 활성화
        idx=i
    )

Validation Analysis:   0%|          | 0/40 [00:00<?, ?it/s]

[000] > UNetV0: 0.9465, UNetV1: 0.9107
[001] > UNetV0: 0.9159, UNetV1: 0.9062
[002] > UNetV0: 0.8009, UNetV1: 0.3720
[003] > UNetV0: 0.5532, UNetV1: 0.3091
[004] > UNetV0: 0.6218, UNetV1: 0.5683
[005] > UNetV0: 0.5525, UNetV1: 0.6175
[006] > UNetV0: 0.7380, UNetV1: 0.6998
[007] > UNetV0: 0.7074, UNetV1: 0.7174
[008] > UNetV0: 0.4448, UNetV1: 0.4511
[009] > UNetV0: 0.4638, UNetV1: 0.5695
[010] > UNetV0: 0.5357, UNetV1: 0.8215
[011] > UNetV0: 0.4966, UNetV1: 0.5662
[012] > UNetV0: 0.4695, UNetV1: 0.4817
[013] > UNetV0: 0.2082, UNetV1: 0.3208
[014] > UNetV0: 0.7823, UNetV1: 0.7491
[015] > UNetV0: 0.4928, UNetV1: 0.5732
[016] > UNetV0: 0.8846, UNetV1: 0.8910
[017] > UNetV0: 0.9441, UNetV1: 0.9103
[018] > UNetV0: 0.9204, UNetV1: 0.9078
[019] > UNetV0: 0.8529, UNetV1: 0.8110
[020] > UNetV0: 0.9403, UNetV1: 0.9319
[021] > UNetV0: 0.9805, UNetV1: 0.9824
[022] > UNetV0: 0.9474, UNetV1: 0.9501
[023] > UNetV0: 0.9450, UNetV1: 0.9558
[024] > UNetV0: 0.9267, UNetV1: 0.9582
[025] > UNetV0: 0.9425, U

In [41]:
# test 데이터
compare_test_dir = os.path.join('./', 'data', 'result', 'comparison_results', 'unetv0,1_test')
os.makedirs(compare_test_dir, exist_ok=True)

for i in tqdm(range(200), desc="Test Inference Progress"):
    img_name = f"00{str(i).zfill(4)}_10.png"
    img_full_path = os.path.join(dir_path, "image_2", img_name)
    res_full_path = os.path.join(compare_test_dir, f"result_{str(i).zfill(3)}.png")

    get_comparison_output(
        unet_v0, unet_v1, val_preproc,
        image_path=img_full_path,
        output_path=res_full_path,
        model_type=['UNetV0','UNetV1'],
        label_path=None, # 테스트는 라벨 없음
        idx=i
    )

Test Inference Progress:   0%|          | 0/200 [00:00<?, ?it/s]

UNetV1(KittiDataset) VS UNeV2(KittiDatasetV2)

In [42]:
# 검증 데이터는 val_dataset.data 리스트를 사용하여 라벨 경로를 함께 넘깁니다.
compare_val_dir = os.path.join('./', 'data', 'result', 'comparison_results', 'unetv1,2_val')
os.makedirs(compare_val_dir, exist_ok=True)

for i in tqdm(range(len(val_dataset_v2.data)), desc="Validation Analysis"):
    img_path, label_path = val_dataset_v2.data[i]
    res_full_path = os.path.join(compare_val_dir, f"val_compare_{str(i).zfill(2)}.png")

    get_comparison_output(
        unet_v1, unet_v2, val_preproc,
        image_path=img_path,
        output_path=res_full_path,
        model_type=['UNetV1','UNetV2'],
        label_path=label_path, # 라벨 전달 -> IoU 및 텍스트 출력 활성화
        idx=i
    )

Validation Analysis:   0%|          | 0/40 [00:00<?, ?it/s]

[000] > UNetV1: 0.9107, UNetV2: 0.9582
[001] > UNetV1: 0.9062, UNetV2: 0.9156
[002] > UNetV1: 0.3720, UNetV2: 0.8812
[003] > UNetV1: 0.3091, UNetV2: 0.4966
[004] > UNetV1: 0.5683, UNetV2: 0.8269
[005] > UNetV1: 0.6175, UNetV2: 0.8397
[006] > UNetV1: 0.6998, UNetV2: 0.7478
[007] > UNetV1: 0.7174, UNetV2: 0.9340
[008] > UNetV1: 0.4511, UNetV2: 0.5041
[009] > UNetV1: 0.5695, UNetV2: 0.4146
[010] > UNetV1: 0.8215, UNetV2: 0.8714
[011] > UNetV1: 0.5662, UNetV2: 0.7465
[012] > UNetV1: 0.4817, UNetV2: 0.7850
[013] > UNetV1: 0.3208, UNetV2: 0.5310
[014] > UNetV1: 0.7491, UNetV2: 0.9190
[015] > UNetV1: 0.5732, UNetV2: 0.8546
[016] > UNetV1: 0.8910, UNetV2: 0.8976
[017] > UNetV1: 0.9103, UNetV2: 0.9404
[018] > UNetV1: 0.9078, UNetV2: 0.9108
[019] > UNetV1: 0.8110, UNetV2: 0.8708
[020] > UNetV1: 0.9319, UNetV2: 0.9621
[021] > UNetV1: 0.9824, UNetV2: 0.9835
[022] > UNetV1: 0.9501, UNetV2: 0.9563
[023] > UNetV1: 0.9558, UNetV2: 0.9763
[024] > UNetV1: 0.9582, UNetV2: 0.9753
[025] > UNetV1: 0.9551, U

In [43]:
# test 데이터
compare_test_dir = os.path.join('./', 'data', 'result', 'comparison_results', 'unetv1,2_test')
os.makedirs(compare_test_dir, exist_ok=True)

for i in tqdm(range(200), desc="Test Inference Progress"):
    img_name = f"00{str(i).zfill(4)}_10.png"
    img_full_path = os.path.join(dir_path, "image_2", img_name)
    res_full_path = os.path.join(compare_test_dir, f"result_{str(i).zfill(3)}.png")

    get_comparison_output(
        unet_v1, unet_v2, val_preproc,
        image_path=img_full_path,
        output_path=res_full_path,
        model_type=['UNetV1','UNetV2'],
        label_path=None, # 테스트는 라벨 없음
        idx=i
    )

Test Inference Progress:   0%|          | 0/200 [00:00<?, ?it/s]

#### UNet ++
FocalLoss, KittiDatasetV2 사용

In [44]:
# # 모델 생성
# unet_pp = Nested_UNet(input_channels=3, output_channels=1).to(device)

# # 손실 함수 및 옵티마이저 설정
# criterion = FocalLoss(alpha=0.5, gamma=2.0)

# train_model(
#     unet_pp, train_loader_v2, val_loader_v2, criterion, 
#     device=device, 
#     num_epochs=epoch, 
#     save_dir=save_dir, 
#     model_type='_unet_pp'  # 모델 타입 명시
# )

In [45]:
model_path = f"{save_dir}/best_unet_pp.pth"
unet_pp = Nested_UNet(input_channels=3, output_channels=1)
unet_pp.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
unet_pp.eval()  # 평가 모드로 전환

Nested_UNet(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (up): Upsample(scale_factor=2.0, mode='bilinear')
  (conv0_0): Unet_block(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (conv1_0): Unet_block(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (conv2_0): Unet_block(
    (conv1): Conv2d(1

In [46]:
dir_path = './data/testing'
save_train_dir = './data/result/unet_pp_train'
save_test_dir = './data/result/unet_pp_test'

os.makedirs(dir_path, exist_ok=True)
os.makedirs(save_train_dir, exist_ok=True)
os.makedirs(save_test_dir, exist_ok=True)

In [47]:
# 검증 데이터는 val_dataset.data 리스트를 사용하여 라벨 경로를 함께 넘깁니다.
for i in tqdm(range(len(val_dataset_v2.data)), desc="Validation Analysis"):
    img_path, label_path = val_dataset_v2.data[i]

    get_output(
        unet_pp, val_preproc,
        image_path=img_path,
        output_path= f'{save_train_dir}/result_{str(i).zfill(3)}.png',
        label_path=label_path,
        im_show = False
    )

Validation Analysis:   0%|          | 0/40 [00:00<?, ?it/s]

In [48]:
for i in tqdm(range(200), desc="Test Inference Progres", unit="img"):
    
    img_name = f"00{str(i).zfill(4)}_10.png"
    img_full_path = os.path.join(dir_path, "image_2", img_name)
    
    res_name = f"result_{str(i).zfill(3)}.png"
    res_full_path = os.path.join(save_test_dir, res_name)

    # 추론 실행
    get_output(
        unet_pp,
        val_preproc,
        image_path=img_full_path,
        output_path=res_full_path
    )

Test Inference Progres:   0%|          | 0/200 [00:00<?, ?img/s]

In [49]:
# test 데이터
compare_test_dir = os.path.join('./', 'data', 'result', 'comparison_results', 'UNetP_test')
os.makedirs(compare_test_dir, exist_ok=True)

for i in tqdm(range(200), desc="Test Inference Progress"):
    img_name = f"00{str(i).zfill(4)}_10.png"
    img_full_path = os.path.join(dir_path, "image_2", img_name)
    res_full_path = os.path.join(compare_test_dir, f"result_{str(i).zfill(3)}.png")

    get_comparison_output(
        unet_v2, unet_pp, val_preproc,
        image_path=img_full_path,
        output_path=res_full_path,
        model_type=['UNetV2','UNet++'],
        label_path=None, # 테스트는 라벨 없음
        idx=i
    )

Test Inference Progress:   0%|          | 0/200 [00:00<?, ?it/s]

In [50]:
# 검증 데이터는 val_dataset.data 리스트를 사용하여 라벨 경로를 함께 넘깁니다.
compare_val_dir = os.path.join('./', 'data', 'result', 'comparison_results', 'UNetP_val')
os.makedirs(compare_val_dir, exist_ok=True)

for i in tqdm(range(len(val_dataset_v2.data)), desc="Validation Analysis"):
    img_path, label_path = val_dataset_v2.data[i]
    res_full_path = os.path.join(compare_val_dir, f"val_compare_{str(i).zfill(2)}.png")

    get_comparison_output(
        unet_v2, unet_pp, val_preproc,
        image_path=img_path,
        output_path=res_full_path,
        model_type=['UNetV2','UNet++'],
        label_path=label_path, # 라벨 전달 -> IoU 및 텍스트 출력 활성화
        idx=i
    )

Validation Analysis:   0%|          | 0/40 [00:00<?, ?it/s]

[000] > UNetV2: 0.9582, UNet++: 0.9560
[001] > UNetV2: 0.9156, UNet++: 0.8686
[002] > UNetV2: 0.8812, UNet++: 0.9393
[003] > UNetV2: 0.4966, UNet++: 0.4883
[004] > UNetV2: 0.8269, UNet++: 0.7912
[005] > UNetV2: 0.8397, UNet++: 0.5719
[006] > UNetV2: 0.7478, UNet++: 0.5931
[007] > UNetV2: 0.9340, UNet++: 0.8657
[008] > UNetV2: 0.5041, UNet++: 0.4916
[009] > UNetV2: 0.4146, UNet++: 0.4975
[010] > UNetV2: 0.8714, UNet++: 0.8775
[011] > UNetV2: 0.7465, UNet++: 0.8561
[012] > UNetV2: 0.7850, UNet++: 0.6443
[013] > UNetV2: 0.5310, UNet++: 0.5453
[014] > UNetV2: 0.9190, UNet++: 0.9028
[015] > UNetV2: 0.8546, UNet++: 0.7817
[016] > UNetV2: 0.8976, UNet++: 0.8965
[017] > UNetV2: 0.9404, UNet++: 0.8411
[018] > UNetV2: 0.9108, UNet++: 0.8422
[019] > UNetV2: 0.8708, UNet++: 0.8707
[020] > UNetV2: 0.9621, UNet++: 0.9488
[021] > UNetV2: 0.9835, UNet++: 0.9779
[022] > UNetV2: 0.9563, UNet++: 0.9607
[023] > UNetV2: 0.9763, UNet++: 0.9745
[024] > UNetV2: 0.9753, UNet++: 0.9692
[025] > UNetV2: 0.9771, U